In [ ]:
# Loading the MNIST dataset
from tensorflow.keras.datasets import mnist

# Loading the MNIST traning and testing sets using the load_data()
(X_train,y_train), (X_test,y_test) = mnist.load_data()

"""
   Data Exploration by checking the dimensions of the training set images(X_train),
   training set labels (y_train), testing set images(X_test) and testing set labels(y_test)
"""
X_train.shape

y_train.shape

X_test.shape

y_test.shape

# Visualizing the digits
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale=2)

import numpy as np

"""Passing a sequence of indice as a numpy array's subscript to select only the array values at that index"""
index = np.random.choice(np.arange(len(X_train)), 24, replace=False)
figure, axes = plt.subplots(nrows=4, ncols=6, figsize=(16, 9))

for item in zip(axes.ravel(),X_train[index], y_train[index]):
    axes, image, target = item
    axes.imshow(image, cmap=plt.cm.gray_r)
    axes.set_xticks([])
    axes.set_yticks([])
    axes.set_title(target)
plt.tight_layout()

In [ ]:
# Data Preparation

X_train = X_train.reshape((60000,28,28,1))

X_test = X_test.reshape((10000,28,28,1))

X_train = X_train.astype('float32')/255

X_test = X_test.astype('float32')/255

from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)

y_train.shape

y_train[0]

y_test = to_categorical(y_test)

y_test.shape

In [ ]:
# Creating a neural network
from tensorflow.keras.models import Sequential

cnn = Sequential()

# Addin g layers to the network
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D

# Adding a convolutional layer
cnn.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu", input_shape=(28, 28, 1)))

cnn.add(MaxPooling2D(pool_size=(2,2)))

# Adding another convolutional layer and maxPooling layer
cnn.add(Conv2D(filters=128, kernel_size=(3,3),activation="relu"))

cnn.add(MaxPooling2D(pool_size=(2,2)))

# Flattening the results
cnn.add(Flatten())

# Adding a dense layer to reduce the number of features
cnn.add(Dense(units=128, activation="relu"))

#Adding another dense layer to produce the output
cnn.add(Dense(units=10, activation="softmax"))

cnn.summary()



In [ ]:
# Visualizing the model's structure
from tensorflow.keras.utils import plot_model
from IPython.display import Image

plot_model(cnn,to_file="convnet.png", show_shapes=True, show_layer_names=True)

Image(filename="convnet.png")

# Compiling the model
cnn.compile(optimizer="adam", loss="categorical_crossentropy",metrics=["accuracy"])


# Training and evaluating the model
cnn.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

# Evaluating the model
loss, accuracy = cnn.evaluate(X_train,y_train)

loss

accuracy

# making predictions
predictions = cnn.predict(X_test)

# Checking what the first sample digit should be
y_test[0]

# Checking the probabilities returned by the predict method for the first sample test.
for index, probability in enumerate(predictions[0]):
    print(f"{index}: {probability:.10%}")

In [ ]:
# locating the incorrect predictions by comparing the index of the largest probability in
# predictions[0] to the index of the element containing 1.0 in y_test[0]
images = X_test.reshape((10000, 28, 28))
incorrect_predictions = []

for i, (p, e) in enumerate(zip(predictions, y_test)):
    predicted, expected = np.argmax(p), np.argmax(e)

    if predicted != expected:
        incorrect_predictions.append(
            (i, images[i], predicted, expected)
        )

len(incorrect_predictions)

# Visualizing incorrect predictions
figure,  axes = plt.subplots(nrows=4, ncols=6,figsize=(16, 12))

for axes, item in zip(axes.ravel(), incorrect_predictions):
    index, image, predicted, expected = item
    axes.imshow(image, cmap=plt.cm.gray_r)
    axes.set_xticks([])
    axes.set_yticks([])
    axes.set_title(f"index:{index}\np:{predicted}; e:{expected}")
plt.tight_layout()

# Displaying the probabilities for several incorrect predictions
def display_probabilities(prediction):
    for index, probability in enumerate(prediction):
        print(f"{index}:{probability}")

display_probabilities(predictions[495])

display_probabilities(predictions[583])

display_probabilities(predictions[625])

# Saving and loading the model
cnn.save("mnist_cnn.h5")

from tensorflow.keras.models import load_model
cnn = load_model("mnist_cnn.h5")